In [877]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.impute import SimpleImputer

In [878]:
raw_train = pd.read_csv("train.csv")
raw_test = pd.read_csv("test.csv")
raw_sample_submission = pd.read_csv("sample_submission.csv")

In [879]:
numeric_cols = list(raw_train.select_dtypes(include=np.number).keys())
object_cols = list(raw_train.select_dtypes(include=np.object_).keys())

In [880]:
print(len(numeric_cols),len(object_cols))
print(len(numeric_cols)+len(object_cols))

38 43
81


In [881]:
raw_train_numeric = raw_train.copy()[numeric_cols[1:-1]]

In [882]:
imputer = SimpleImputer(strategy="mean")

In [883]:
imputed_cols = imputer.fit_transform(raw_train_numeric)

In [884]:
raw_train_numeric[numeric_cols[1:-1]] = imputed_cols

In [885]:
raw_train_numeric["SalePrice"] = raw_train.copy()["SalePrice"]

In [886]:
train_num, test_num = train_test_split(raw_train_numeric,test_size=0.2,random_state=12)

In [887]:
model1 = LinearRegression()
model1.fit(train_num.drop("SalePrice",axis = 1),train_num["SalePrice"])

LinearRegression()

In [888]:
pred1 = model1.predict(test_num.drop("SalePrice",axis = 1))
rmse1 = root_mean_squared_error(test_num["SalePrice"],pred1)

In [889]:
test_set_num = raw_test[numeric_cols[1:-1]]

In [890]:
test_imputer = SimpleImputer(strategy="mean")
test_imputer_cols = test_imputer.fit_transform(test_set_num)

In [891]:
test_set_num[numeric_cols[1:-1]] = test_imputer_cols

C:\Users\Sachin_2\AppData\Local\Temp\ipykernel_8964\3262949261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set_num[numeric_cols[1:-1]] = test_imputer_cols


In [892]:
test_pred1 = model1.predict(test_set_num)

In [893]:
test_set_num["SalePrice"] = test_pred1

C:\Users\Sachin_2\AppData\Local\Temp\ipykernel_8964\3336705832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set_num["SalePrice"] = test_pred1


In [894]:
df_test = pd.DataFrame({
    "Id":raw_test["Id"].values,
    "SalePrice":test_set_num["SalePrice"].values
})

In [895]:
df_test.to_csv("submission1_new.csv",index=False)

In [896]:
(raw_train.isna().sum() > 0).sum()

np.int64(19)

In [897]:
raw_train[numeric_cols[1:]] = raw_train_numeric

In [898]:
raw_train_obj = raw_train.copy()[object_cols]

In [899]:
imputer_obj = SimpleImputer(strategy="most_frequent")

In [900]:
obj_cols = imputer_obj.fit_transform(raw_train_obj)

In [901]:
raw_train_obj[object_cols] = obj_cols

In [902]:
one_hot_obj = OneHotEncoder()

In [903]:
one_hot_obj_cols = one_hot_obj.fit_transform(raw_train_obj[object_cols])

In [904]:
#Creating category coloumns
category = []
for i in list(one_hot_obj.categories_): 
        for j in i:
            category.append(j)

In [905]:
raw_train_obj_updated = pd.DataFrame(one_hot_obj_cols.toarray(),columns=category)

In [906]:
raw_train.drop(columns=object_cols,axis=1,inplace=True)

In [907]:
raw_train_updated = pd.concat([raw_train,raw_train_obj_updated],axis=1)

# TEST DATASET

In [908]:
imp_test = SimpleImputer(strategy="most_frequent")

In [909]:
imp_test_cols = imputer_obj.transform(raw_test[object_cols])

In [910]:
raw_test[object_cols] = imp_test_cols

In [911]:
oh_test = OneHotEncoder()

In [912]:
oh_test_cols = one_hot_obj.transform(raw_test[object_cols])

In [913]:
#Creating category coloumns
category_test = []
for i in list(one_hot_obj.categories_): 
        for j in i:
            category_test.append(j)

In [914]:
raw_test_obj = pd.DataFrame(oh_test_cols.toarray(),columns=category_test)

In [915]:
raw_test.drop(columns=object_cols,axis=1,inplace=True)
raw_test_updated = pd.concat([raw_test,raw_test_obj],axis=1)

In [916]:
oh_test_num = SimpleImputer(strategy="mean")

In [917]:
raw_test_updated[numeric_cols[:-1]] = oh_test_num.fit_transform(raw_test_updated[numeric_cols[:-1]])

# Training the model2

In [918]:
model2 = LinearRegression()

In [919]:
model2.fit(raw_train_updated.drop(["Id","SalePrice"],axis=1),raw_train_updated["SalePrice"])

LinearRegression()

In [920]:
pred2 = model2.predict(raw_test_updated.drop("Id",axis=1))

In [921]:
submission2 = pd.DataFrame({
    "Id" : raw_test_updated["Id"],
    "SalePrice" : pred2
})

In [930]:
id = np.array(submission2.Id.values,dtype = int)

In [932]:
submission2["Id"] = id

In [934]:
submission2.to_csv("submission2.csv",index=False)